In [1]:
from tianshou_ppo import tianshou_ppo_train
from ortools_scheduler import ORtools_scheduler
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
all_instances = ["ft06", "la01", "la06", "la11", "la21", "la31", "la36", "orb01", "yn1", "swv01", "swv06", 
                "swv11", "swv12", "swv13", "swv14", "swv15"]


test_instances = ["ft06"]

In [3]:
random_rate = 0.5
cv = 0.2
epochs = 10
n = 10                  # 每张图几个点   

instances = all_instances.copy()        

In [4]:
%%capture hide_output
directory = f"figs_no_future_infomation/p{random_rate}cv{cv}epochs{epochs}_more_instances"
if not os.path.exists(directory):
    os.makedirs(directory)

ret = {}
output = ""
ortools_mean, ortools_std, policy_mean, policy_std = [], [], [], []
ortools_on_original, policy_on_original = [], []
optimal_mean, optimal_std = [], []
ortools_300s_optimal_rate = []
current_instances = []

for instance in instances:
    makespan, policy = tianshou_ppo_train(instance_name=instance, max_epoch=epochs)
    ret[instance] = makespan

    scheduler = ORtools_scheduler(instance)
    scheduler.optimize()
    obj_val = scheduler.obj_val

    policy_vals, ortools_vals, optimal_vals, if_optimals = [], [], [], []
    for _ in range(n):
        times = scheduler.shifted_time_(random_rate=random_rate, cv=cv)
        policy_val = scheduler.policy_makespan('ppo', policy, shifted_time=times)
        ortools_val = scheduler.compute_makespan(shifted_time=times)        # 静态调度面对工时波动

        if_optimal, optimal_val = scheduler.get_optimal_of_new_time_mat(times)

        policy_vals.append(policy_val)
        ortools_vals.append(ortools_val)
        optimal_vals.append(optimal_val)
        if_optimals.append(int(if_optimal))

    plt.plot(policy_vals, color='g', label='policy')
    plt.plot(ortools_vals, color='r', label='ortools')
    plt.plot(optimal_vals, color='blue', label='ortools_300s')
    policy_vals, ortools_vals, optimal_vals = np.array(policy_vals), np.array(ortools_vals), np.array(optimal_vals)

    plt.hlines(np.mean(ortools_vals), -2, n+2, linestyles='dotted', colors='r')
    plt.hlines(np.mean(policy_vals), -2, n+2, linestyles='dotted', colors='g')
    plt.hlines(np.mean(optimal_vals), -2, n+2, linestyles='dotted', colors='blue')
    scatter_x = np.where(if_optimals)
    scatter_y = np.array(optimal_vals)[scatter_x]
    plt.scatter(scatter_x, scatter_y, color='blue')


    output += f"Instance : {instance}\n"
    output += f"random rate={random_rate}, cv={cv}, epochs={epochs}"
    output += f"ortools makespan on 原始算例 is {obj_val}, policy makespan on 原始算例 is {makespan}\n"
    output += f"mean of makespan when time mat varies, ortools:{np.mean(ortools_vals)}, policy:{np.mean(policy_vals)}\n"
    output += f"std of makespan when time mat varies, ortools:{np.std(ortools_vals)}, policy:{np.std(policy_vals)}\n"

    ortools_mean.append(np.mean(ortools_vals))
    ortools_std.append(np.std(ortools_vals))
    policy_mean.append(np.mean(policy_vals))
    policy_std.append(np.std(policy_vals))
    optimal_mean.append(np.mean(optimal_vals))
    optimal_std.append(np.std(optimal_vals))
    ortools_on_original.append(obj_val)
    policy_on_original.append(makespan)
    ortools_300s_optimal_rate.append(np.mean(if_optimals))
    current_instances.append(instance)

    plt.xlabel('trial')
    plt.ylabel('makespan')
    plt.title(f"random_rate={random_rate},cv={cv},instance={instance}")
    plt.legend()
    plt.savefig(f"{directory}/policy_vs_ortools_{instance}.png")
    plt.clf()

    
df = pd.DataFrame()
# 将每个列添加到 DataFrame 中
df['instance'] = current_instances
df['ortools_mean'] = ortools_mean
df['policy_mean'] = policy_mean
df['optimal_mean'] = optimal_mean
df['ortools_std'] = ortools_std
df['policy_std'] = policy_std
df['optimal_std'] = optimal_std
df['ortools_on_original'] = ortools_on_original
df['policy_on_original'] = policy_on_original
df['ortools_300s_optimal_rate'] = ortools_300s_optimal_rate

if os.path.exists(f"{directory}/data.csv"):
    df2 = pd.read_csv(f"{directory}/data.csv")
    df2 = df2.append(df, ignore_index=True)
    df2.to_csv(f"{directory}/data.csv", index=False)
else:
    df.to_csv(f"{directory}/data.csv", index=False)

In [5]:
print(output)

Instance : swv14
random rate=0.5, cv=0.2, epochs=10ortools makespan on 原始算例 is 3115.0, policy makespan on 原始算例 is 3506
mean of makespan when time mat varies, ortools:4136.6, policy:3575.0
std of makespan when time mat varies, ortools:145.253709074846, policy:176.2367725532898
Instance : swv15
random rate=0.5, cv=0.2, epochs=10ortools makespan on 原始算例 is 3092.0, policy makespan on 原始算例 is 3501
mean of makespan when time mat varies, ortools:4148.7, policy:3586.6
std of makespan when time mat varies, ortools:186.50150133443967, policy:146.46719769286227



In [6]:
ret

{'swv14': 3506, 'swv15': 3501}